In [1]:
import time
import numpy as np
import pandas as pd
import requests
import xlsxwriter

In [2]:
def weatherHist(airportIndex, year):
    a = 'http://api.wunderground.com/history/airport/'
    b = airportIndex +'/'+ str(year)+'/1/1/CustomHistory.html?dayend=31&monthend=12&yearend='
    c = str(year)+'&format=1'
    url = a+b+c
    page = requests.get(url).text
    data = page.split("<br />")
    dic = {}
    for i in data[1:-1]:
        item = i.split(",")
        dic[str(item[0])[1:]] = [str(a) for a in item[1:]]
    df = pd.DataFrame.from_dict(dic, orient = "index")
    colnames = [str(name) for name in data[0].split(",")]
    df.columns = colnames[1:]
    return df

In [3]:
xl_file = pd.ExcelFile('zip_code_database.xls')

In [4]:
file2 = xl_file.parse('Sheet2')

In [5]:
codes = list(file2.loc[:,'ICAO'])

In [6]:
codes = [str(x) for x in codes if str(x) != 'nan']
codes = [str(x) for x in codes if str(x) != 'none']

In [7]:
def weatherRecord20(code):
    frames = []
    for year in np.arange(20)+1980:
        name = 'df'+str(year)
        try:
            name = weatherHist(code, year)
            frames.append(name)
        except ValueError:
            return "No data"
        time.sleep(10.0)
    return pd.concat(frames)    

In [8]:
def saveDF(code, df):
    name = 'histData/'+code+'.xlsx'
    writer = pd.ExcelWriter(name, engine='xlsxwriter')
    df.to_excel(writer,'Sheet1')
    writer.save()

In [9]:
def nowGetData(codeList):
    dic = {}
    for code in codeList:
        print code
        df = weatherRecord20(code)
        if type(df) != str:
            saveDF(code, df)
            dic[code] = df
    return dic

In [10]:
# weatherDataWA = nowGetData(codes)

KBLI
KFHR
KPSC
KCLM
KPUW
KBFI
KSEA
KGEG
KALW
KEAT
KYKM
KORS
KPAE
KRNT
KSFF
KAWO
KPWT
KBVS
KCLS
KDEW
KELN
KEPH
KHQM
KKLS
KMWH
KOLM
KOMK
KPLU
KUIL
KRLD
KSHN
KTIW
KTDO
KVUO
KDLS
KESW
KOKH
KPUL
KNRA
KNUW
KNOW
KSKA
KTCM
KGRF
KFCT


In [11]:
print len(codes)

45


In [13]:
print weatherDataWA.keys()

['KBFI', 'KPUW', 'KPSC', 'KESW', 'KALW', 'KTDO', 'KVUO', 'KPWT', 'KMWH', 'KPAE', 'KEPH', 'KSEA', 'KHQM', 'KUIL', 'KGRF', 'KTIW', 'KSKA', 'KEAT', 'KOLM', 'KTCM', 'KYKM', 'KBLI', 'KGEG', 'KAWO', 'KSFF', 'KCLM', 'KDLS', 'KRLD']


In [14]:
completeDF = pd.concat(weatherDataWA.values(), keys=weatherDataWA.keys())

In [15]:
completeDF.head()

Max TemperatureF Mean TemperatureF Min TemperatureF  \
KBFI 1980-7-19               70                66               62   
     1980-7-18               80                70               59   
     1980-7-15               77                68               60   
     1980-7-14               68                62               57   
     1980-7-17               77                68               59   

               Max Dew PointF MeanDew PointF Min DewpointF Max Humidity  \
KBFI 1980-7-19             64             60            56           93   
     1980-7-18             61             57            55           90   
     1980-7-15             61             59            58           93   
     1980-7-14             63             60            58          100   
     1980-7-17             59             55            53           83   

                Mean Humidity  Min Humidity  Max Sea Level PressureIn  \
KBFI 1980-7-19             82            70                     30.15   
     1980-7-18             66            47                     30.08   
     1980-7-15             77            54                     30.16   
     1980-7-14             91            81                     30.12   
     1980-7-17             66            48                     30.23   

                     ...        Max VisibilityMiles  Mean VisibilityMiles  \
KBFI 1980-7-19       ...                          8                     7   
     1980-7-18       ...                         20                    12   
     1980-7-15       ...                         15                     8   
     1980-7-14       ...                          8                     5   
     1980-7-17       ...                         20                    14   

                Min VisibilityMiles  Max Wind SpeedMPH  Mean Wind SpeedMPH  \
KBFI 1980-7-19                    4                 12                   7   
     1980-7-18                    7                 12                   5   
     1980-7-15                    7                 15                   8   
     1980-7-14                    1                 10                   5   
     1980-7-17                    7                 12                   5   

                Max Gust SpeedMPH PrecipitationIn  CloudCover    Events  \
KBFI 1980-7-19                               0.00           7       Fog   
     1980-7-18                               0.00           4             
     1980-7-15                               0.00           7             
     1980-7-14                               0.00           8  Fog-Rain   
     1980-7-17                 18            0.00           5             

                WindDirDegrees  
KBFI 1980-7-19             165  
     1980-7-18             328  
     1980-7-15             205  
     1980-7-14             141  
     1980-7-17             348  

[5 rows x 22 columns]

In [16]:
print len(completeDF)

203621


In [18]:
saveDF('CompleteWeatherHist1980_99', completeDF)